In [ ]:
import datetime
import json
import requests
import time

HOST = "http://192.168.4.5:5000"
PROMPT = "Where can I read the latest news?"
ITERATIONS = 10
TOKENS = 1000

def get_model():
    endpoint = f"{HOST}/api/v1/model"
    response = requests.request("GET", endpoint, headers={}, data={})
    response = response.json()
    
    return response['result']

def make_llm_call():
    endpoint = f"{HOST}/api/v1/generate"

    payload = json.dumps({
        "prompt": PROMPT,
        "max_new_tokens": TOKENS,
        "preset": "simple-1"
    })

    response = requests.request(
        "POST", 
        endpoint, 
        headers={
            'Content-Type': 'application/json'
        }, 
        data=payload
    )

    if response.status_code != 200:
        print("Error: Failed to fetch LLM response.")
        return
    
    response = response.content.decode(response.encoding)
    decoded_response = json.loads(response)
    decoded_response = decoded_response['results'][0]['text']

    return decoded_response

def main():
    api_responses = []
    MODEL = get_model()

    print(f"#########################################")
    print(f"HOST: {HOST}")
    print(f"MODEL: {MODEL}")
    print(f"PROMPT: {PROMPT}")
    print(f"ITERATIONS: {ITERATIONS}")
    print(f"#########################################")
    print()

    # Call Ooba API and save responses in the list
    for i in range(ITERATIONS):
        print(f"> Running iteration {i + 1}", end="")

        start_time = time.time()
        llm_response = make_llm_call()
        end_time = time.time()
        processing_time = round(end_time - start_time, 2)

        print(f" → {processing_time} seconds")
        print(llm_response)
        print()
        print()

        data = {
            "_id": i,
            "model_name": MODEL,
            "user_prompt": PROMPT,
            "llm_response": llm_response,
            "processing_time": processing_time
        }

        api_responses.append(data)

    # Save api_responses to a JSON file
    filename = f"{MODEL}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
    with open(filename, "w") as f:
        json.dump(api_responses, f, indent=2)
        print()
        print(f"💾 Saved results to {filename}")

if __name__ == "__main__":
    main()
